# My first project - using vaults

When creating a project, we start defining the infrastructure layout via two files:

  - ansible.cfg
  - inventory


We're going to describe the following setup:

  - direct access to client machines (eg. no bastion)
  - one `inventory` file
  - no host_key_check
  - no retry files
  - client machine username/password is `root:root`
  - no public key is installed by default



In [ ]:
cd /notebooks/exercise-01/

## Preparation

The first steps include creating:

 - all secret files (ssh identity, vault pin file)
 - ansible.cfg (the deployment descriptor) 
 - and the inventory.

In [ ]:
# At first create a proper ssh key for the project
! rm id_ansible.pub id_ansible -rf

Now create a new ssh key for the project:

- the default name is `id_ansible`;
- the algorithm is `ed25519`, which is the most efficient one. For further information on ed25519 see [RFC8032](https://datatracker.ietf.org/doc/html/rfc8032).




In [ ]:
! test -f id_ansible || ssh-keygen -q -t ed25519 -f id_ansible  -N ''

In [ ]:
# Use the `lookup` plugin to create a password file.
!ansible localhost -m copy -a "content='{{ lookup('password', '/dev/null length=20 chars=ascii_letters,digits') }}' dest=$PWD/.pin"

In [ ]:
# Let's show the password :)
! cat .pin

In [ ]:
# then prepare a deployment descriptor referencing the .pin file
!cat ansible.cfg

#### Exercise

What is the meaning of the `-q` and `-N` options of `ssh-keygen`?

Hint: open a [terminal](/terminals/man) and use `man ssh-keygen`.


#### Exercise

 - use ansible docs or the web to comment properly all the [ansible.cfg](/edit/notebooks/exercise-01/ansible.cfg) params

In [ ]:
# And in the end, the inventory. 
!cat inventory

- use the `copy` module to generate the `vault.yaml` file with the following content

```yaml
root_password: <a random password generated via the lookup plugin>
```

In [ ]:
# Solution
!ansible localhost -mcopy \
    -a "content='root_password: {{lookup('password','/dev/null')}}' dest=$PWD/vault.yaml"



- use the ansible_connection=docker method specified in `inventory.docker` to copy the ssh key to the docker containers. Without this, the ssh key will not be available to the docker containers and you cannot access them via ssh.

In real infrastructures, the ssh keys are usually provisioned at boot.

[![](https://mermaid.ink/img/pako:eNp1U1Fr2zAQ_itCDyMFK8xKnLmmFArdw6CFsr3NCkORLo2pLQVJbpsl-e87W3bTdBQ_6O6-T3efPll7qqwGWlDGmDChCjUU5Mb4alUDUdYYUKGyhjQQNlZ7YXreo5PbDbn7KYwwvl3FVFv1BK6MC1nLVbGWLGbLd7St1CXBgkpL7B9kZcClSO_Yql1BB_ETxM8hn04mPx7IuMFAeLHuib1UDvTFBeK8x_lnOBgtTBSl_WQSIyEM7vlkx0Auy9uR68E998EozT76D-ddDpN8StiUXROVdtKGmJ_ZJqPZf05mI-g3rDLPYIJ1uxIz0ljd4o3EgTUOWY7K3hEH69-45zfQC_rQmzB2fcDKgaDSL8Tz_7v2lFg8kNG5k4cdPMSku1TsovCgjB2u3ug0oQ24RlYaf7Q9-kYEDRtoQNACQw1r2dZBUGGOSG23Wgb4riucTovgWkiobIP9tTNqzCPntpLoYUPxmLXH6laa39Y2IwlTWuzpKy3SLJ_OUz6bZXnOZxn_xhO6owW_XEwX2eUiz_LZfJ7hd0zo377D12mWUOg13McH0r-T4z9VnhsC?type=png)](https://mermaid.live/edit#pako:eNp1U1Fr2zAQ_itCDyMFK8xKnLmmFArdw6CFsr3NCkORLo2pLQVJbpsl-e87W3bTdBQ_6O6-T3efPll7qqwGWlDGmDChCjUU5Mb4alUDUdYYUKGyhjQQNlZ7YXreo5PbDbn7KYwwvl3FVFv1BK6MC1nLVbGWLGbLd7St1CXBgkpL7B9kZcClSO_Yql1BB_ETxM8hn04mPx7IuMFAeLHuib1UDvTFBeK8x_lnOBgtTBSl_WQSIyEM7vlkx0Auy9uR68E998EozT76D-ddDpN8StiUXROVdtKGmJ_ZJqPZf05mI-g3rDLPYIJ1uxIz0ljd4o3EgTUOWY7K3hEH69-45zfQC_rQmzB2fcDKgaDSL8Tz_7v2lFg8kNG5k4cdPMSku1TsovCgjB2u3ug0oQ24RlYaf7Q9-kYEDRtoQNACQw1r2dZBUGGOSG23Wgb4riucTovgWkiobIP9tTNqzCPntpLoYUPxmLXH6laa39Y2IwlTWuzpKy3SLJ_OUz6bZXnOZxn_xhO6owW_XEwX2eUiz_LZfJ7hd0zo377D12mWUOg13McH0r-T4z9VnhsC)

In [ ]:
# Provision the ssh keys using the docker_connector
!cat inventory.docker
!ansible docker -i inventory.docker -m shell \
       -a "echo 'root:{{root_password}}' | chpasswd" \
       -e@vault.yaml 

## Gotta ping 'em all

Our Ansible environment is all set, but we still need to check if we can reach all the hosts.

Let's ping all hosts, eventually adjusting the ip ranges in the [inventory](/edit/notebooks/exercise-01/inventory).

Open a [terminal](/terminals/man) and use `docker inspect` to retrieve the container IPs.

In [ ]:
# Let's get an error: root_password is UNDEFINED. 
!ansible -m ping all

## Vaults

We now ensure that everything is encrypted and we're able to connect to some hosts

In [ ]:
# A vault is just a yaml file containing a dictionary of secrets.
#  We can put here as many information as we want, but for now
#  just set a `root_password` generated using the lookup plugin.
!cat vault.yaml

In [ ]:
# We need to encrypt it. 
!ansible-vault encrypt vault.yml


#### Exercise

  - which secret is used to encrypt vault.yml ?
  - where is it specified?
  - what happens if you try to re-encrypt the vault.yml

In [ ]:
# And show the anatomy of the vault.
!cat vault.yaml

In [ ]:
# Can we decrypt it?
!ansible-vault view vault.yaml

In [ ]:
!ansible -m ping all -e@vault.yaml

#### Exercise

 - How can you pass the vault password file from the command line?
 - Run ansible in verbose mode and check how ansible pass the password to ssh: does it use some helper program?

In [ ]:
# Write answers here

## SSH Authentication

We want to switch from password to ssh authentication. Create a playbook to install ssh keys: it reads the password from vault.yaml


In [28]:
!cat copy-key.yml

cat: copy-key.yml: File o directory non esistente


In [ ]:
!cat id_ansible.pub

In [ ]:
!ansible-playbook copy-key.yml

#### Exercise

Comment out the ansible_password field  [in `inventory` here](/edit/notebooks/exercise-01/inventory)

  - guess the expected output without running ansible
  
  

In [ ]:
# Running in debug mode we can see all ssh arguments injected via anisble. Discuss the DEBUG output       
!sed -i 's/ansible_password/#ansible_password/' inventory
!ansible -vvv -m ping all

In [ ]:
# Use this cell for the exercise

#### Exercise

Run ansible in verbose mode to se all the injected ssh argument. If the output is too verbose, reduce it either with:

- `--limit ipaddress` to contact only one node
- host indexing/subscript eg: `all[0]` 

# Discuss vaults for:
- common secrets to be shared on a repo
- private secrets to reside on local PC